In [3]:
import csbsingleenv

In [4]:
env = csbsingleenv.CodersStrikeBackSingle()

In [5]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    rewards.append(reward)
print(sum(rewards))

-10100


In [6]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

2024-04-24 14:32:42,342	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [7]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-04-24 14:32:44,203	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [8]:
config = (PPOConfig()
          .environment(csbsingleenv.CodersStrikeBackSingle)
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

tuner.fit()

2024-04-24 14:32:59,445	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1525567) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1525567) 2024-04-24 14:33:02,916	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.411704937023903}, 'default_policy': {'total_loss': 7.411704937023903, 'policy_loss': -0.04951420044556244, 'vf_loss': 7.447755664396388, 'vf_loss_unclipped': 5122334.1540012, 'vf_explained_var': -0.0001593113326823025, 'entropy': 4.551092605092633, 'mean_kl_loss': 0.02991890234596616, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10183.0, 'episode_reward_mean': -10102.96, 'episode_len_mean': 102.96, 'episode_media': {}, 'episodes_this_iter': 39, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean':